<a href="https://colab.research.google.com/github/stutir1412/STUTI-TechnicalAssignment/blob/main/Task_3_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3: NLP Training

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


To first download the dataset from the provided link

In [2]:
!wget https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttrain.bio
!wget https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttest.bio

--2023-11-21 00:01:55--  https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttrain.bio
Resolving groups.csail.mit.edu (groups.csail.mit.edu)... 128.30.2.44
Connecting to groups.csail.mit.edu (groups.csail.mit.edu)|128.30.2.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 759757 (742K)
Saving to: ‘restauranttrain.bio.1’

restauranttrain.bio 100%[===================>] 741.95K  1.24MB/s    in 0.6s    

2023-11-21 00:01:56 (1.24 MB/s) - ‘restauranttrain.bio.1’ saved [759757/759757]

--2023-11-21 00:01:56--  https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttest.bio
Resolving groups.csail.mit.edu (groups.csail.mit.edu)... 128.30.2.44
Connecting to groups.csail.mit.edu (groups.csail.mit.edu)|128.30.2.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155722 (152K)
Saving to: ‘restauranttest.bio.1’

restauranttest.bio. 100%[===================>] 152.07K   455KB/s    in 0.3s    

2023-11-21 00:01:57 (455 KB/s) - ‘r

Pre-Processing Dataset

- Reverse the tag and the column column in both the test and train files.

- Replace the tab seperator to space.

- Change the example seperator to newline

In [3]:
# Convert the content of the 'restauranttrain.bio' file to a new file 'train.txt'
# by swapping the first and second columns and saving it to the specified path.

!awk '{print $2,$1}' restauranttest.bio > "/content/drive/My Drive/WalnutAI/test.txt"
!awk '{print $2,$1}' restauranttrain.bio > "/content/drive/My Drive/WalnutAI/train.txt"

# Replace all tab characters ('\t') with a space (' ') in the 'test.txt' file.
!sed -i 's/\t/ /g' '/content/drive/My Drive/WalnutAI/test.txt'

# Replace the beginning of a line followed by a space ('^ $') with a newline character ('\n')
# in the 'test.txt' file. This is intended to handle empty lines.
!sed -i 's/^ $/\n/g' '/content/drive/My Drive/WalnutAI/test.txt'

# Replace all tab characters ('\t') with a space (' ') in the 'train.txt' file.
!sed -i 's/\t/ /g' '/content/drive/My Drive/WalnutAI/train.txt'

# Replace the beginning of a line followed by a space ('^ $') with a newline character ('\n')
# in the 'train.txt' file. This is intended to handle empty lines.
!sed -i 's/^ $/\n/g' '/content/drive/My Drive/WalnutAI/train.txt'

In [4]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/WalnutAI/train.txt',sep=' ', header=None)

Create a labels document

In [5]:
with open('/content/drive/My Drive/WalnutAI/labels.txt','w') as fout:
  for tag in df[1].unique():
    fout.write(tag+'\n')

Implement Spacy NLP Libraray

In [6]:
# Install spaCy and download a spaCy model
!pip install spacy
!python -m spacy download en_core_web_sm

2023-11-21 00:02:12.316967: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-21 00:02:12.317027: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-21 00:02:12.317063: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-21 00:02:12.325380: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 00:02:13.447937: W tensorflow/c

In [8]:
# Import necessary libraries
import spacy
from spacy.training.example import Example
import random

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Load training data
with open('/content/drive/My Drive/WalnutAI/train.txt') as f:
    training_data = f.readlines()

# Load the labels
with open('/content/drive/My Drive/WalnutAI/labels.txt') as label_file:
    labels = label_file.read().splitlines()

# Add labels to the NER component
ner = nlp.get_pipe('ner')
for label in labels:
    ner.add_label(label)

# Define the training loop
def train_spacy_ner(training_data, iterations=5):
    # Train the model for a given number of iterations
    for epoch in range(iterations):
        random.shuffle(training_data)
        losses ={}
        for line in training_data:
            parts = line.split()
            if len(parts) > 1:
                text = parts[1:]
                entities = []
                start = 0
                for word in text:
                    entities.append((start, start + len(word), parts[0]))
                    start += len(word) + 1
                example = Example.from_dict(nlp.make_doc(' '.join(text)), {"entities": entities})
                ner.update([example], drop=0.5, losses=losses )

        print(f'Epoch {epoch + 1}/{iterations} - Losses: {losses["ner"]}')

    return nlp

# Train the NER model
trained_nlp = train_spacy_ner(training_data)





Epoch 1/5 - Losses: 106076.41537637149
Epoch 2/5 - Losses: 103432.16096246311
Epoch 3/5 - Losses: 103597.24967763793
Epoch 4/5 - Losses: 103589.07117387516
Epoch 5/5 - Losses: 103326.35299982602


To Save the Trained Model

In [9]:
trained_nlp.to_disk('/content/drive/My Drive/WalnutAI/ner_model')

To evaluate the model

In [13]:
loaded_nlp = spacy.load('/content/drive/My Drive/WalnutAI/ner_model')

# Test the model with a sample text
text = "any 50s style diners around" #Type input text here
doc = loaded_nlp(text)

# Extract named entities and their labels in dictionary format
named_entities = {ent.text: ent.label_ for ent in doc.ents}
print(named_entities)

{'50s': 'CARDINAL'}


Currently, after testing with 3 sentences, I observed that my named entity recognition model accuracy is really poor which is why it gives out wrong output enitity. I believe by adopting the following techniques the model can improve
- To train it for more number of iterations (around 20) to allow the model have more exposure to the training data. Currently, due to the large dataset I could only train for 5 epoch as it took extremely long time. Additionly, running on GPU can help improve training time.
- Dropout rate can be adjusted to lower it to around 0.3. A higher dropout rate can help prevent overfitting, but it might slow down convergence.
- Experiment with different spaCy model architectures or use a larger pre-trained model for better performance. Such as - using en_core_web_md or en_core_web_lg instead of en_core_web_sm.
